In [ ]:
# Q1. Install MLflow
# What's the version that you have?
# A: mlflow, version 2.22.0


# Q2. Download and preprocess the data
# How many files were saved to OUTPUT_FOLDER?
#A: 4


# Q3. Train a model with autolog
# What is the value of the min_samples_split parameter:
# A: 2

# Q4. Launch the tracking server locally
# In addition to backend-store-uri, what else do you need to pass to properly configure the server?
# A: default-artifact-root


# Q5. Tune model hyperparameters
# What's the best validation RMSE that you got?
# A: 5.335


# Q6. Promote the best model to the model registry
# What is the test RMSE of the best model?
# 5.567


In [ ]:
# conda create -n exp-tracking-env python=3.9

# conda activate exp-tracking-env

#  conda deactivate

#  mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root /workspaces/mlops-zoomcamp2025/homeworks/hw2/artifacts

In [ ]:
python preprocess_data.py --raw_data_path /workspaces/mlops-zoomcamp2025/homeworks/hw2/data --dest_path ./output

In [ ]:
import os
import pickle
import click
import mlflow

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, root_mean_squared_error


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


@click.command()
@click.option(
    "--data_path",
    default="./output",
    help="Location where the processed NYC taxi trip data was saved"
)
def run_train(data_path: str):
    mlflow.sklearn.autolog()

    with mlflow.start_run():
        X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
        X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)

        rmse =  root_mean_squared_error(y_val, y_pred)
        # mean_squared_error(y_val, y_pred,squared=False)
        mlflow.log_metric("rmse", rmse)
    
    


if __name__ == '__main__':
    mlflow.set_tracking_uri("sqlite:///mlflow.db")
    mlflow.set_experiment("hw2-experiment")   

    run_train()
